## Statistics and Data Analysis, Homework 3
# Differential Gene Expression in Acute Myocardial Infraction
Gabriel Mazor, Yam Libman

## The Data Set
1. Dataset record in NCBI:
   https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE66360
2. Published Paper: Muse et al, Sci Rep 2017
   https://www.nature.com/articles/s41598-017-12166-0 

### Pre-processing the dataset

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [98]:
pathA = "AMI_GSE66360_series_matrix.csv"
data = pd.read_csv(pathA, encoding='ISO-8859-1', header=None)
pd.set_option('display.max_rows', 100)
data.head(70)

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"!Series_title\t""A Whole Blood Molecular Signat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"!Series_geo_accession\t""GSE66360""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"!Series_status\t""Public on Feb 28 2015""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"!Series_submission_date\t""Feb 27 2015""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"!Series_last_update_date\t""Dec 27 2017""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"!Series_summary\t""Despite the significant redu...",CVD still accounts for a third of all deaths ...,dyslipidemia,diabetes mellitus,cigarette smoking,inactivity) remain the focus of primary preve...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"!Series_summary\t""We isolated circulating endo...",and measured gene expression using the HG-133...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"!Series_overall_design\t""Circulating endotheli...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"!Series_type\t""Expression profiling by array""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"!Series_contributor\t""Eric",R,"Kramer""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
df = data.drop(data.index[:59]).reset_index(drop=True)
df = df.transpose()
df.columns=df.iloc[0]
df=df.drop(df.index[0])
df.head()

,Class,ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
1,H,GSM1620819,5.866377893,8.22579012,6.17973231,6.179477974,2.792047952,9.290410779,6.998285145,5.348943925,...,13.257325,13.266235,14.57123,14.276568,4.504659,3.139334,2.747015,2.651554,3.413429,3.140612
2,H,GSM1620820,4.948736331,7.525865369,6.628137025,6.58288903,2.69571445,9.462275035,6.558214949,5.410884095,...,13.59386,13.517013,14.738834,14.341233,4.409226,3.039961,2.638606,2.643615,3.399063,3.132691
3,H,GSM1620821,5.148384393,7.764636529,5.859151477,6.602134766,2.580181122,9.116776316,6.851622539,5.254073031,...,13.372759,13.416583,14.627181,14.318568,4.310076,3.220109,2.495261,2.626028,3.53997,3.698396
4,H,GSM1620822,5.064418945,7.198461482,5.974135101,6.545904723,2.712123845,9.134584186,7.357827486,5.242139865,...,13.490226,13.494827,14.754124,14.363297,4.439311,3.263782,2.544199,2.504958,3.396225,3.638612
5,H,GSM1620823,5.017832878,7.831450258,6.793079158,5.911542321,2.723686912,9.582343511,7.283872601,5.219483996,...,13.236785,13.330192,14.548389,14.263677,4.562883,3.138524,2.50663,2.609707,3.43472,3.577649


In [100]:
df.dtypes

0
Class              object
ID_REF             object
1007_s_at          object
1053_at            object
117_at             object
                    ...  
AFFX-ThrX-5_at     object
AFFX-ThrX-M_at     object
AFFX-TrpnX-3_at    object
AFFX-TrpnX-5_at    object
AFFX-TrpnX-M_at    object
Length: 54677, dtype: object

In [101]:
for col in df.columns[2:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dtypes

0
Class               object
ID_REF              object
1007_s_at          float64
1053_at            float64
117_at             float64
                    ...   
AFFX-ThrX-5_at     float64
AFFX-ThrX-M_at     float64
AFFX-TrpnX-3_at    float64
AFFX-TrpnX-5_at    float64
AFFX-TrpnX-M_at    float64
Length: 54677, dtype: object

# Analysis

#### a. High level description of the data and some pre-processing

1. How many genes profiled?

In [88]:
df.head()

,Class,ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
1,H,GSM1620819,5.866378,8.225790,6.179732,6.179478,2.792048,9.290411,6.998285,5.348944,...,13.257325,13.266235,14.571230,14.276568,4.504659,3.139334,2.747015,2.651554,3.413429,3.140612
2,H,GSM1620820,4.948736,7.525865,6.628137,6.582889,2.695714,9.462275,6.558215,5.410884,...,13.593860,13.517013,14.738834,14.341233,4.409226,3.039961,2.638606,2.643615,3.399063,3.132691
3,H,GSM1620821,5.148384,7.764637,5.859151,6.602135,2.580181,9.116776,6.851623,5.254073,...,13.372759,13.416583,14.627181,14.318568,4.310076,3.220109,2.495261,2.626028,3.539970,3.698396
4,H,GSM1620822,5.064419,7.198461,5.974135,6.545905,2.712124,9.134584,7.357827,5.242140,...,13.490226,13.494827,14.754124,14.363297,4.439311,3.263782,2.544199,2.504958,3.396225,3.638612
5,H,GSM1620823,5.017833,7.831450,6.793079,5.911542,2.723687,9.582344,7.283873,5.219484,...,13.236785,13.330192,14.548389,14.263677,4.562883,3.138524,2.506630,2.609707,3.434720,3.577649


In [91]:
print(df.shape[1]-2, "genes")


54675 genes


2. How many samples (subjects/patients) in total?

In [93]:
print(df.shape[0], "samples")

99 samples


3. How many samples in each class?

In [95]:
print(df['Class'].value_counts())

H    50
M    49
Name: Class, dtype: int64


4. If there are missing values, then remove the entire row (gene) from the data matrix. How many rows left now?

In [97]:
nullColumns = df.isnull().sum(axis=0)
columns_with_null = nullColumns[nullColumns > 0].index
df = df.drop(columns_with_null, axis=1)
print(df.shape[1]-2, "genes after removing columns with null values")


54675 genes after removing columns with null values


5. Pick 20 genes at random. Draw 20 pair boxplots in one figure comparing expression levels of each of these genes in the two classes M and H.

In [42]:
randomGenes = df.sample(n=20)
# Select 20 random genes
random_genes = df.sample(n=20)

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 8))

# Iterate over the selected genes
for i, gene in enumerate(random_genes['ID_REF']):
    # Get the expression levels for the gene in classes M and H
    expression_M = df.loc[df['ID_REF'] == gene, col[col == 'M'].index]
    expression_H = df.loc[df['ID_REF'] == gene, col[col == 'H'].index]
    
    # Create a list of expression levels for the two classes
    data = [expression_M.values.flatten(), expression_H.values.flatten()]
    
    # Plot the boxplot for the gene
    ax.boxplot(data, positions=[i*2, i*2+1], widths=0.6, showfliers=False)

# Set the x-axis labels
ax.set_xticks(range(0, 40, 2))
ax.set_xticklabels(random_genes['ID_REF'], rotation=90)

# Set the y-axis label
ax.set_ylabel('Expression Level')

# Set the plot title
ax.set_title('Pair Boxplots of Randomly Selected Genes')

# Show the plot
plt.show()


##

#### b. WRS for differential expression (DE)

1. Consider some gene, g. Under the null model (which assumes that for g there is no M vs H DE), what is the expected sum of ranks of g’s expression levels measured for samples labeled M?

2. Denote this sum of ranks by RS(g). What is the maximal value, c, that RS(g) can take?

3. Under the null model, what is the probability of RS(g) = c? (Provide a formula for this and explain it)

4. Under the null model, what is the probability of RS(g) = c-1? what is the probability of RS(g) = c-2? (Provide formulas and explain them)

5. Draw a histogram of the values of RS(g) in the dataset. Here g ranges over all genes in the data (after the clean-up). Compute the IQR for this distribution and present it on the plot with the histogram.

#### c. Differential Expression
The purpose is to determine the statistical significance of differential expression (DE) observed for each gene in H vs M.
Evaluate the DE in both one-sided directions for every gene, using both Student t-test and WRS test.
Report the number of genes overexpressed in M vs H (M > H) at a p-value better (≤) than 0.07 and separately genes underexpressed in M vs H (M < H) at a p-value better than 0.07.
For both directions use both a Student t-test and a WRS test.

#### d. Correlations
Select the 80 most significant genes from each one of the onesided
WRS DE lists you computed in 3c. Generate a set of 160
genes, D, which is the union of the above two sets.

1. Compute Spearman 𝜌 correlations in all pairs within D (160 choose 2 numbers). Represent the correlation matrix as a 160x160 heatmap.

2. Under a NULL model that assumes that genes are pairwise independent, what is the expected value for 𝜌?

3. Now compute the Spearman 𝜌 correlations in all 80 choose 2 pairs from the overexpressed genes in D. Present your results on a histogram. What is the average value you observed?
Compare it to the value in the above section. Explain.

4. What can you report about co-expression of genes in D (coexpression is inferred from the correlation of the expression levels of genes, across a set of samples)?

5. What can you say about how many co-expressed pairs we would observe (in the entire dataset) at FDR=0.05? 
   Explain your answer.

6. What would have been advantages and disadvantages of computing co-expression for all genes in the study rather than only for genes in D?

#### e. Plot and Conclusions of the DE and correlation analysis

1. Construct the DE overabundance plots (blue and green lines as shown in class) for M vs H overexpression (higher expression levels in M) using WRS and t-test using the results you had computed in Section 3c.  
State, for each comparison, the number of genes, k, at whichwe observe with an FDR threshold of:  
a) 𝜏 = 0.05  
b) 𝜏 = 0.01  
c) 𝜏 = 0.005    
  If these events are not observed at any k>0, then make that statement.

2. For any given gene, g, consider the following set of p-values:
𝑆(𝑔) = -𝑝(𝜆)|
𝑝(𝜆) 𝑖𝑠 𝑎 𝑊𝑅𝑆 𝑝 − 𝑣𝑎𝑙𝑢𝑒
𝑓𝑜𝑟 𝑜𝑣𝑒𝑟𝑒𝑥𝑝𝑟𝑒𝑠𝑠𝑖𝑜𝑛 𝑖𝑛 𝑀
𝑎𝑓𝑡𝑒𝑟 𝑠𝑤𝑎𝑝𝑝𝑖𝑛𝑔 𝑜𝑛𝑒 𝑙𝑎𝑏𝑒𝑙
E
(In the above definition 𝜆 represents a label swap – there are
99 such swaps).
Let
𝑝!(𝑔) = max
"
𝑆(𝑔)
In words: 𝑝!(𝑔) represents the maximum p-value that could
have been obtained for g, assuming a single labeling error.
a) For all genes, compute 𝑝!(𝑔).
b) Run the FDR procedure with 𝜏 = 0.05 using 𝑝!(𝑔)
(instead of the original p-values as computed in
section 3e1a above). How many genes can you report
now?
c) What is the intersection of the genes you can report
above with the results of 3e1a. Explain.
Comment: The process you ran in this section yields a set
of genes called Robust Differentially Expressed Genes
(RDEG).

3. Select any 3 differentially expressed genes, from D (which
was defined in 3d), and produce a graphical representation of
their expression patterns that demonstrates the observed DE.

4. Heatmap
Draw a heatmap representation of the expression values of
the genes in D (from 3d), across the entire cohort (all
samples). Order the genes and the samples to produce the
maximal visual effect.